In [5]:
import yfinance as yf
import pandas as pd
from IPython.display import display, HTML
import requests
import numpy as np

def b_print(df): 
    
    display(HTML(df.to_html(index=True)))
    df = None

info = [
'Net Income',           #Lucro Líquido
'Operating Cash Flow',  #Fluxo de Caixa Operacional
'Total Debt',           #Dívida Líquida
'Stockholders Equity',  #Patrimônio Líquido
'Current Assets',       #Ativo Circulante
'Current Liabilities',  #Passivo Circulante
'Gross Profit',         #Lucro Bruto
'Total Revenue',        #Receita Total
'Total Assets']         #Ativo Total


In [3]:
def piotroski_score(data):
  """
  Calcula o score de Piotroski para uma empresa com base em um DataFrame com dados od yahoo finance

  Args:
    data: DataFrame com index=['Net Income','Operating Cash Flow', etc.] e colunas=['2023', '2022'].

  Returns:
    int: Score de Piotroski (0 a 9).
  """

  score = 0

  # Rentabilidade
  score += int(data.loc['Net Income', '2023-12-31'] > 0)  # Lucro Líquido positivo
  score += int(data.loc['Operating Cash Flow', '2023-12-31'] > 0) # Fluxo de Caixa Operacional positivo
  score += int(data.loc['Net Income', '2023-12-31'] > data.loc['Net Income', '2022-12-31'])  # ROA superior ao ano anterior (aproximação) 
  score += int(data.loc['Operating Cash Flow', '2023-12-31'] > data.loc['Net Income', '2023-12-31'])  # Fluxo de Caixa Operacional superior ao Lucro Líquido

  # Alavancagem e Liquidez
  divida_patrimonio_2023 = data.loc['Total Debt', '2023-12-31'] / data.loc['Stockholders Equity', '2023-12-31']
  divida_patrimonio_2022 = data.loc['Total Debt', '2022-12-31'] / data.loc['Stockholders Equity', '2022-12-31']
  score += int(divida_patrimonio_2023 < divida_patrimonio_2022)  # Dívida/Ativo inferior ao ano anterior
  
  liquidez_corrente_2023 = data.loc['Current Assets', '2023-12-31'] / data.loc['Current Liabilities', '2023-12-31']
  liquidez_corrente_2022 = data.loc['Current Assets', '2022-12-31'] / data.loc['Current Liabilities', '2022-12-31']
  score += int(liquidez_corrente_2023 > liquidez_corrente_2022) # Liquidez corrente superior ao ano anterior
  # Emissões de ações: Não disponível no Yahoo Finance

  # Eficiência Operacional
  margem_bruta_2023 = data.loc['Gross Profit', '2023-12-31'] / data.loc['Total Revenue', '2023-12-31']
  margem_bruta_2022 = data.loc['Gross Profit', '2022-12-31'] / data.loc['Total Revenue', '2022-12-31']
  score += int(margem_bruta_2023 > margem_bruta_2022) # Margem bruta superior ao ano anterior

  giro_ativo_2023 = data.loc['Total Revenue', '2023-12-31'] / data.loc['Total Assets', '2023-12-31']
  giro_ativo_2022 = data.loc['Total Revenue', '2022-12-31'] / data.loc['Total Assets', '2022-12-31']
  score += int(giro_ativo_2023 > giro_ativo_2022)  # Giro do ativo superior ao ano anterior

  return score

In [1]:
import pyarrow.parquet as pq
# Abrir o arquivo Parquet
arquivo_parquet = pq.ParquetFile("data/yf_financial_data_Y.parquet")
tabela_parquet = arquivo_parquet.read()
df = tabela_parquet.to_pandas()

In [3]:
df.head(2)

AALR3                            \
                                   2023         2022         2021   
Tax Effect Of Unusual Items  4432920.00 -1152656.000  1091374.000   
Tax Rate For Calcs                 0.34        0.061        0.191   

                                         ABCB4                            \
                                    2020  2023      2022      2021  2020   
Tax Effect Of Unusual Items  29330440.00  0.00  0.000000  0.000000  0.00   
Tax Rate For Calcs                  0.34  0.34  0.098805  0.049481  0.34   

                                    ABEV3                ...          WIZC3  \
                                     2023          2022  ...           2022   
Tax Effect Of Unusual Items -1.386500e+07 -7.701000e+08  ... -196491.629184   
Tax Rate For Calcs           5.000000e-03  3.400000e-01  ...       0.220282   

                                                                YDUQ3  \
                                      2021          2020         2023   
Tax Effect Of Unusual Items -250133.065707 -1.145998e+07 -12826840.00   
Tax Rate For Calcs                0.346445  3.528210e-01         0.34   

                                                                     \
                                     2022         2021         2020   
Tax Effect Of Unusual Items  2.583032e+06  43053520.00  52912500.00   
Tax Rate For Calcs           3.642690e-01         0.34         0.34   

                                  ZAMP3                    
                                   2022        2021  2020  
Tax Effect Of Unusual Items -7810140.00 -2503080.00  0.00  
Tax Rate For Calcs                 0.34        0.34  0.34  

[2 rows x 690 columns]

In [8]:
url = 'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/SI_Acoes.csv'
pd.set_option('display.max_columns', None)
fundsSI = pd.read_csv(url,sep=';' , decimal=',' ,thousands ='.' )
fundsSI[fundsSI['ROA']>0]

,TICKER,PRECO,DY,P/L,P/VP,P/ATIVOS,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,EV/EBIT,DIVIDA LIQUIDA / EBIT,DIV. LIQ. / PATRI.,PSR,P/CAP. GIRO,P. AT CIR. LIQ.,LIQ. CORRENTE,ROE,ROA,ROIC,PATRIMONIO / ATIVOS,PASSIVOS / ATIVOS,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA,VPA,LPA,PEG Ratio,VALOR DE MERCADO
1,ABCB4,23.07,6.84,5.64,0.96,0.10,25.50,17.06,15.50,5.12,5.12,NaN,NaN,0.87,-2.67,-0.10,0.05,16.95,1.73,NaN,0.10,0.91,0.11,18.70,17.80,1.506057e+07,24.15,4.09,-0.31,5.466116e+09
2,ABEV3,12.03,6.07,12.57,2.10,1.37,50.30,22.59,18.29,10.18,9.43,-0.75,-0.15,2.30,51.13,-1.89,1.11,16.71,10.94,18.63,0.65,0.34,0.60,10.72,15.72,3.590053e+08,5.73,0.96,0.83,1.895186e+11
5,AESB3,9.23,NaN,35.21,1.24,0.28,35.54,28.78,4.90,6.00,15.96,9.96,2.06,1.73,17.01,-0.34,1.11,3.53,0.80,5.04,0.23,0.72,0.16,NaN,NaN,2.069151e+07,7.43,0.26,-0.52,5.555789e+09
6,AFLT3,6.98,NaN,14.37,1.68,1.41,57.31,45.58,44.72,14.10,13.72,-0.38,-0.05,6.43,14.43,-1.75,1.98,11.71,9.80,10.57,0.84,0.16,0.22,24.96,19.61,8.023230e+03,4.15,0.49,-0.19,4.403312e+08
7,AGRO3,25.72,12.49,10.30,1.20,0.75,30.65,20.17,20.86,10.65,12.50,1.85,0.21,2.15,3.68,-1.22,2.14,11.68,7.31,7.59,0.63,0.37,0.35,27.22,15.22,7.141002e+06,21.40,2.50,-0.24,2.641018e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,WHRL4,4.65,8.05,19.61,2.73,0.74,16.27,8.15,3.24,7.80,7.25,-0.71,-0.25,0.64,11.71,-2.58,1.10,13.92,3.75,17.70,0.27,0.73,1.16,12.32,-3.95,4.737778e+04,1.70,0.24,-0.57,7.131973e+09
615,WIZC3,7.41,6.01,12.18,2.58,0.52,58.84,37.07,9.26,3.04,3.31,0.26,0.22,1.13,-20.34,-0.67,0.90,21.18,4.24,22.73,0.20,0.59,0.46,12.37,-9.41,3.567407e+06,2.87,0.61,-0.32,1.184913e+09
616,WLMM3,23.50,1.70,8.10,1.28,1.04,12.97,7.04,4.99,5.75,6.09,-0.61,-0.14,0.40,2.37,-2.36,4.54,15.77,12.80,15.79,0.81,0.19,2.56,32.16,NaN,3.456000e+03,18.40,2.90,-1.18,9.986176e+08
617,WLMM4,30.70,1.43,10.58,1.67,1.35,12.97,7.04,4.99,7.51,6.09,-0.61,-0.14,0.53,3.09,-3.09,4.54,15.77,12.80,15.79,0.81,0.19,2.56,32.16,NaN,9.950222e+04,18.40,2.90,-1.55,9.986176e+08
